In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.7/112.6 GB disk)


In [2]:
!pip install optuna
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="bSbuPJyAkojNGnXNQwV6")
project = rf.workspace("detectionclassificationgarbage").project("detection-garbage-jkww2")
version = project.version(5)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Detection-Garbage-5 in yolov11:: 100%|██████████| 45348/45348 [00:07<00:00, 6294.83it/s]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
db_path = "sqlite:////content/drive/MyDrive/SIC/capstone/optuna_rtdetr.db"

In [ ]:
import optuna
from ultralytics import RTDETR
import os

def objective_rtdetr(trial):
    lr = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

    model = RTDETR("rtdetr-l.pt")

    model.train(
        data="/content/Detection-Garbage-5/data.yaml",
        epochs=2,
        imgsz=640,
        lr0=lr,
        weight_decay=weight_decay,
        optimizer="AdamW",
        project="optuna_rtdetr",
        name=f"trial_{trial.number}",
        exist_ok=True,
        verbose=False
    )

    metrics = model.val()
    map50 = metrics.box.map50

    return map50 



In [7]:
study = optuna.create_study(
    study_name="rtdetr_tuning",
    storage=db_path,
    direction="maximize",
    load_if_exists=True
)
study.optimize(objective_rtdetr, n_trials=1)

print("Best trial:")
print(study.best_trial.params)


[I 2025-06-27 02:03:31,995] Using an existing study with name 'rtdetr_tuning' instead of creating a new one.


100%|██████████| 63.4M/63.4M [00:01<00:00, 41.8MB/s]


Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Detection-Garbage-5/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=5.236145672630982e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=trial_4, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plot

100%|██████████| 755k/755k [00:00<00:00, 46.0MB/s]

Overriding model.yaml nc=80 with nc=8
WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5, 

...

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 16, len(boxes) = 6235. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [01:16<00:00,  1.58it/s]


                   all       1935       6235      0.279      0.293       0.16     0.0983
Speed: 0.3ms preprocess, 35.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to optuna_rtdetr/trial_4


[I 2025-06-27 02:55:12,603] Trial 4 finished with value: 0.16022844655661758 and parameters: {'lr0': 5.236145672630982e-05, 'weight_decay': 0.0002203030589865087}. Best is trial 3 with value: 0.38686239869670547.


Best trial:
{'lr0': 0.00023408388126935296, 'weight_decay': 1.5677089901579907e-05}


In [8]:
study = optuna.load_study(
    study_name="rtdetr_tuning",
    storage=db_path
)

best_trial = study.best_trial

print("✅ Giá trị mục tiêu tốt nhất:", best_trial.value)
print("✅ Bộ tham số tối ưu:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

✅ Giá trị mục tiêu tốt nhất: 0.38686239869670547
✅ Bộ tham số tối ưu:
  lr0: 0.00023408388126935296
  weight_decay: 1.5677089901579907e-05


In [9]:
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()